In [1]:
# !pip install -qU pip wheel
# !pip install -q -r ../requirements.txt
import sys
sys.path.append('..')
from ballitoreproject import *
from ballitoreproject.topicmodels import *


In [2]:
# get all data (metadata plus letters)
df = get_data()
df

,sender,recipient,date,location from,location to,notes,box,datetime,year,decade,is_journal,txt,supplemental_ids,num_letters,num_words
id,,,,,,,,,,,,,,,
mss4-b1-f7-001-176,William Penn,Unknown 1.1,2/10/1690,London,,copy; p. 2,1,1690-02-10,1690,1690,False,,mss4-b1-f7-001-177,1,0
mss4-b1-f3-005,Abraham Shackleton I,Richard Shackleton,1744,,Dublin,extract,1,1744-07-13,1744,1740,False,"P.7\t“Keep thy mind as quiet as possible, so t...",,1,37
mss4-b1-f7-001-3,Sophia Hume,Anne Hyam,6/1/1748,Philadelphia,,; p. 6,1,1748-06-01,1748,1740,False,,mss4-b1-f7-001-30,1,0
mss4-b1-f3-005-006,Abraham Shackleton I,Richard Shackleton,1748,,Dublin,,1,1748-07-13,1748,1740,False,"“We are very desirous of seeing thee at home, ...",,1,173
mss4-b1-f7-001-15,Sophia Hume,J. P.,5/19/1750,London,,,1,1750-05-19,1750,1750,False,,,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
consensus_text_90345943,,,,,,,14,NaT,0,0,True,"was no company here to day, and\neven Charlie ...",,1,74
consensus_text_90345944,,,,,,,14,NaT,0,0,True,"dinner, Papa, Ned, and the\nrest came out to t...",,1,69
consensus_text_90345945,,,,,,,14,NaT,0,0,True,[deletion]16[/deletion] Papa did not go to tow...,,1,71


In [3]:
df.sender.value_counts().head(25)

sender
                               2206
Richard Shackleton              349
Sarah Leadbeater Barrington     200
William Rayner                   99
Mary Shackleton Leadbeater       74
Margaret Shackleton Grubb        67
Juliet Smith                     51
Selina Barrington                46
Samuel Haughton                  44
Lydia J. Leadbeater Fisher       44
Edward Barrington                42
Thomas Fisher                    42
Benjamin Haughton                42
Sarah Shackleton                 34
Sarah Tuke Grubb                 34
S. Hancock                       32
William Leadbeater               31
Simeon Wilkinson                 27
James J. Fisher                  27
Frances Smith                    25
Elizabeth Leadbeater             23
M. J. Barrington                 23
Elizabeth Dale                   22
Emily Malone                     22
Elizabeth Clibborn               21
Name: count, dtype: int64

In [4]:
# decide on a sample!
df_smpl = df.query('txt!=""')  # only texts (for topic modeling)
df_smpl = df_smpl.query('sender!=""')  # only rows for which we have sender
# df_smpl = df_smpl.query('sender!="" & recipient!=""')  # only rows for which we have sender and recipient
# df_smpl = df_smpl.query('is_journal==True')  # just the journals
# df_smpl = df_smpl.query('sender=="Mary Shackleton Leadbeater"')  # just MSL
# df_smpl = df_smpl.query('sender=="Mary Shackleton Leadbeater" or sender=="Sarah Shackleton"')  # just MSL or SS
df_smpl

,sender,recipient,date,location from,location to,notes,box,datetime,year,decade,is_journal,txt,supplemental_ids,num_letters,num_words
id,,,,,,,,,,,,,,,
mss4-b1-f3-005,Abraham Shackleton I,Richard Shackleton,1744,,Dublin,extract,1,1744-07-13,1744,1740,False,"P.7\t“Keep thy mind as quiet as possible, so t...",,1,37
mss4-b1-f3-005-006,Abraham Shackleton I,Richard Shackleton,1748,,Dublin,,1,1748-07-13,1748,1740,False,"“We are very desirous of seeing thee at home, ...",,1,173
mss4-b1-f3-007,Richard Shackleton,Abraham Shackleton I,11/1/1755,Dublin,,,1,1755-11-01,1755,1750,False,P. 25\t\t\t\t\t\t“Dublin 1st of Eleventhmonth ...,,1,73
mss4-b1-f3-008,Abraham Shackleton I,Richard Shackleton,6/2/1760,London,,,1,1760-06-02,1760,1760,False,"P. 53\t\t\t\t\t\t“London, 2nd of Sixthmonth 17...",,1,43
mss4-b1-f3-008-010,Abraham Shackleton I,Richard Shackleton,9/18/1762,Cooladine,,,1,1762-09-18,1762,1760,False,To the memorable visit paid to the meetings fo...,,1,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mss4-b12-f63-001,Cousin,GL,,,,"essay on ""The Beauties of Nature""",12,NaT,0,0,False,The Beauties of Nature\nWhat is more beautiful...,,1,164
mss4-b12-f63-002,WL?,,,,,,12,NaT,0,0,False,On seeing your Bror yesterday I presume\nyou a...,,1,120
mss4-b12-f63-003,WL?,,,,,includes poem draft in ML's handwriting,12,NaT,0,0,False,Thursday - I received your Enclosure\nand than...,,1,90


In [5]:
from pprint import pprint
pprint(dict(df_smpl.iloc[0]))

{'box': 1,
 'date': 1744,
 'datetime': Timestamp('1744-07-13 00:00:00'),
 'decade': 1740,
 'is_journal': False,
 'location from': '',
 'location to': 'Dublin',
 'notes': 'extract',
 'num_letters': 1,
 'num_words': 37,
 'recipient': 'Richard Shackleton',
 'sender': 'Abraham Shackleton I',
 'supplemental_ids': '',
 'txt': 'P.7\t“Keep thy mind as quiet as possible, so that thou mayst have '
        'benefit of this little tour. I desire thou mayst have thy eye to Him '
        'that hath preserved, in some degree, him that writeth, and is his '
        'chiefest joy. Next to that, my joy is that mine may walk acceptably '
        'before Him. Durable riches, I know, and honours, are in his '
        'right-hand, which He dispenses impartially, in his own fit time.”',
 'year': 1744}


In [6]:
tokenize(df_smpl.iloc[0].txt)

['keep',
 'thy',
 'mind',
 'quiet',
 'possible',
 'thou',
 'mayst',
 'benefit',
 'little',
 'tour',
 'desire',
 'thou',
 'mayst',
 'thy',
 'eye',
 'hath',
 'preserved',
 'degree',
 'writeth',
 'chiefest',
 'joy',
 'next',
 'joy',
 'mine',
 'may',
 'walk',
 'acceptably',
 'durable',
 'riches',
 'know',
 'honours',
 'right',
 'hand',
 'dispenses',
 'impartially',
 'fit',
 'time']

In [7]:
stopwords = get_stopwords()
stopwords|={'us'}
# stopwords

In [8]:
# start a topic model
mdl = tp.LDAModel(k=50)

# loop over dataframe
for id,row in df_smpl.iterrows():
    # get tokens (string of letter to )
    tokens = tokenize(row.txt, stopwords=stopwords)
    # add document
    mdl.add_doc(tokens)

# train model
mdl.train(1000, show_progress=True)

# show summary of results
mdl.summary(topic_word_top_n=25)

Iteration: 100%|██████████| 1000/1000 [00:09<00:00, 102.53it/s, LLPW: -8.628503]


<Basic Info>
| LDAModel (current version: 0.12.7)
| 1931 docs, 453113 words
| Total Vocabs: 28715, Used Vocabs: 28715
| Entropy of words: 7.97182
| Entropy of term-weighted words: 7.97182
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 1000, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.63270
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 50 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 1273551343 (random seed)
| trained in version 0.12.7
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.0282

In [47]:
topic_term_dists = pd.DataFrame([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
topic_term_dists.columns = list(mdl.used_vocabs)
topic_term_dists

,thy,dear,thee,thou,may,much,would,well,love,time,...,obligatns,rems,anyone's,chickens,ballylea,obscuring,granny,wel,snooks,boundaries
0,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,...,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06,3.537131e-06
1,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,...,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06,4.391454e-06
2,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,2.736275e-04,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,...,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06,2.709183e-06
3,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,...,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06,1.694011e-06
4,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,...,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,3.557403e-04,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06,3.522181e-06
5,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,...,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06,1.436338e-06
6,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,7.498710e-04,1.496748e-06,...,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06,1.496748e-06
7,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,...,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06,4.145679e-06
8,2.002981e-02,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,...,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06,4.450081e-06
9,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,4.237818e-03,1.033362e-06,1.033362e-06,...,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06,1.033362e-06


In [48]:
top_topics_for_abolition_keyword = topic_term_dists.sort_values('abolition',ascending=False).head(5)
top_topics_for_abolition_keyword

,thy,dear,thee,thou,may,much,would,well,love,time,...,obligatns,rems,anyone's,chickens,ballylea,obscuring,granny,wel,snooks,boundaries
2,0.000003,0.000003,0.000003,0.000003,0.000003,0.000274,0.000003,0.000003,0.000003,0.000003,...,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
29,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
38,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,...,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
21,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.005964,0.001630,0.000005,...,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005
49,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,...,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005


In [67]:
top_abolition_topic_ids = top_topics_for_abolition_keyword.index

top_topics_for_abolition_keyword.T.sort_values(top_abolition_topic_ids[0],ascending=False).head(100).index

Index(['one', 'men', 'people', 'country', 'made', 'america', 'england', 'city',
       'man', 'war', 'united', 'weeks', 'soldiers', 'king', 'france', 'years',
       'sea', 'shot', 'fall', 'inhabitants', 'devil', 'wonder', 'number',
       'story', 'miles', 'irish', 'captain', 'ought', 'fell', 'gives',
       'history', 'santon', 'ship', 'purpose', 'immediately', 'roman',
       'places', 'hold', 'thousand', 'asked', 'struck', 'paris', 'name',
       'slave', 'fox', 'killed', 'british', 'fire', 'carried', 'took',
       'princess', 'thro'', 'beat', 'son', 'island', 'newyork', 'punishment',
       'europe', 'boat', 'military', 'hundred', 'wounded', 'protestants',
       'barsisa', 'numbers', 'unfortunate', 'fate', 'congress', 'arms',
       'trade', 'paper', 'canius', 'obtained', 'widow', 'although',
       'published', 'fish', 'miserable', 'ear', 'nobody', 'bank', 'kings',
       'procession', 'isle', 'rebellion', 'sailing', 'clergy', 'dead',
       'remarkable', 'boats', 'bone', 'wash

In [68]:
top_topics_for_abolition_keyword.T.sort_values(top_abolition_topic_ids[1],ascending=False).head(100).index

Index(['que', 'dans', 'est', 'sister', 'une', 'magistrate', 'nicholas',
       'vergennes', 'adieu', 'count', 'pas', 'mon', 'non', 'bring', 'les',
       'bancroft', 'diogenes', 'qu'il', 'pollux', 'des', 'broder', 'edict',
       'mea', 'françois', 'billet', 'mois', 'marie', 'vous', 'point', 'chere',
       'horace', 'records', 'affirmation', 'opened', 'declared', 'examined',
       'charged', 'souhaite', 'tis', 'berur', 'comme', 'tous', 'sera', 'nous',
       'dialogue', 'grace', 'salisbury', 'wished', 'notables', 'pedigree',
       'porte', 'philosophers', 'forces', 'notre', 'manly', 'aussi', 'jour',
       'langue', 'par', 'avec', 'repetition', 'approaching', 'grand', 'davies',
       'consequent', 'counties', 'sligo', 'apon', 'l'adroite', 'netete', 'mes',
       'voyage', 'thread', 'entre', 'senr', 'misrepresent', 'virgil', 'pence',
       'brimstone', 'holmes', 'nales', 'ady', 'tallow', 'elsey', 'bonnets',
       'record', 'suspicion', 'archbishop', 'elle', 'soient', 'gascoigne',


In [69]:
doc_topic_scores_df=pd.DataFrame([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_scores_df.index = df_smpl.index
doc_topic_scores_df


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
id,,,,,,,,,,,,,,,,,,,,,
mss4-b1-f3-005,0.000630,0.000492,0.001061,0.001369,0.000649,0.002671,0.003286,0.001051,0.000594,0.025767,...,0.004766,0.038019,0.012147,0.072750,0.000521,0.003052,0.000575,0.027020,0.001043,0.000474
mss4-b1-f3-005-006,0.000156,0.000122,0.000263,0.000340,0.000161,0.000664,0.017396,0.000261,0.000148,0.000873,...,0.001184,0.070236,0.069337,0.117549,0.000129,0.000758,0.000143,0.006711,0.000259,0.000118
mss4-b1-f3-007,0.000350,0.000273,0.000589,0.013115,0.000360,0.001483,0.001824,0.000583,0.000330,0.001952,...,0.002646,0.070529,0.006744,0.139231,0.000289,0.001695,0.000319,0.002647,0.000579,0.000263
mss4-b1-f3-008,0.000555,0.000434,0.000936,0.001208,0.000573,0.002357,0.002899,0.000927,0.000524,0.003101,...,0.004204,0.053172,0.089240,0.064181,0.000460,0.002693,0.000507,0.023838,0.000920,0.000418
mss4-b1-f3-008-010,0.000151,0.005438,0.000254,0.000327,0.000155,0.005960,0.000786,0.000251,0.000142,0.000841,...,0.001140,0.025054,0.199776,0.150417,0.000125,0.006051,0.000137,0.011782,0.000249,0.000113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mss4-b12-f63-001,0.000165,0.000128,0.000277,0.000358,0.046698,0.000698,0.000859,0.000275,0.000155,0.000919,...,0.001246,0.120441,0.055519,0.001567,0.000136,0.000798,0.000150,0.001246,0.000273,0.000124
mss4-b12-f63-002,0.000221,0.000173,0.000373,0.000481,0.000228,0.000938,0.008970,0.000369,0.000209,0.001235,...,0.001674,0.146230,0.082429,0.025554,0.000183,0.001072,0.047099,0.157998,0.000366,0.000167
mss4-b12-f63-003,0.000289,0.000226,0.000487,0.112942,0.041139,0.042067,0.001508,0.000482,0.000273,0.001613,...,0.032818,0.099128,0.117888,0.002750,0.000239,0.001401,0.000264,0.114502,0.000478,0.000218


In [70]:
top_abolition_topic_documents = doc_topic_scores_df[top_abolition_topic_ids[0]].sort_values(ascending=False).head(25).index
top_abolition_topic_documents

Index(['mss4-b7-f8-007', 'mss4-b7-f8-001', 'mss4-b7-f8-005', 'mss4-b7-f8-010',
       'mss4-b7-f8-006', 'mss4-b7-f8-008', 'mss4-b7-f8-009',
       'mss4-b11-f19-001', 'mss4-b1-f4-001-28', 'mss4-b7-f8-002',
       'mss4-b10-f12-002', 'mss4-b6-f3-005', 'mss4-b6-f4-009',
       'mss4-b12-f47-005', 'mss4-b11-f41-002', 'mss4-b6-f4-004',
       'mss4-b3-f15-004-02-03', 'mss4-b11-f51-001', 'mss4-b7-f8-012',
       'mss4-b8-f14-001', 'mss4-b6-f4-011', 'mss4-b6-f4-002',
       'mss4-b12-f10-001', 'mss4-b10-f28-001', 'mss4-b5-f50-005'],
      dtype='object', name='id')

In [71]:
df_smpl.loc[top_abolition_topic_documents[0]]

sender                                                 William Rayner
recipient                                  Mary Shackleton Leadbeater
date                                                        2/27/1772
location from                                                        
location to                                                 Ballitore
notes                                                                
box                                                                 7
datetime                                          1772-02-27 00:00:00
year                                                             1772
decade                                                           1770
is_journal                                                      False
txt                 Bl Hall 27/2 mo: 1772\nDear Molly,\nI beg remi...
supplemental_ids                                                     
num_letters                                                         1
num_words           

In [72]:
print(df_smpl.loc[top_abolition_topic_documents[4]].txt)

Dialogue 1.
A Polyphemus & Neptune
P. O Father, what things have [underline][/underline]
[underline] I suffer'd from that wicked [/underline]
[underline] guest of mine, who when he had [/underline]
[underline] made me drunk & blinded me
[underline] attacking me heavy with sleep. [/underline]
[underline] N. But who dare do that, Polyphemus. [/unclear] [/underline]
P. Indeed at first he called [/underline]
[underline] himself Nobody, but after he had [/underline]
[underline] fled , & was out of the reach of weapons [/underline]
he said his name was Ulysses.
[underline] N. I know whom thou sayest, that
Ithacan, he was sailing back from
[underline] [unclear] Troy [/unclear] but how cou'd he do that for he [/underline]
[underline] was not strong enough. P. I caught in [/underline]
my cave as I was returning from pasture
[underline] a good many fellows lying in [/underline]
[underline] wait no doubt for the cattle. But after that [/underline]
[underline] I put the stopper to the mouth of the

In [46]:
from IPython.display import HTML

topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency,
    start_index=0,
    sort_topics=False
)
pyLDAvis.save_html(prepared_data, 'lda.html')
HTML(filename='lda.html')

In [99]:
all_data=df.join(doc_topic_scores_df)
all_data

,sender,recipient,date,location from,location to,notes,box,datetime,year,decade,...,40,41,42,43,44,45,46,47,48,49
id,,,,,,,,,,,,,,,,,,,,,
mss4-b1-f7-001-176,William Penn,Unknown 1.1,2/10/1690,London,,copy; p. 2,1,1690-02-10,1690,1690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mss4-b1-f3-005,Abraham Shackleton I,Richard Shackleton,1744,,Dublin,extract,1,1744-07-13,1744,1740,...,0.004766,0.038019,0.012147,0.072750,0.000521,0.003052,0.000575,0.027020,0.001043,0.000474
mss4-b1-f7-001-3,Sophia Hume,Anne Hyam,6/1/1748,Philadelphia,,; p. 6,1,1748-06-01,1748,1740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mss4-b1-f3-005-006,Abraham Shackleton I,Richard Shackleton,1748,,Dublin,,1,1748-07-13,1748,1740,...,0.001184,0.070236,0.069337,0.117549,0.000129,0.000758,0.000143,0.006711,0.000259,0.000118
mss4-b1-f7-001-15,Sophia Hume,J. P.,5/19/1750,London,,,1,1750-05-19,1750,1750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
consensus_text_90345943,,,,,,,14,NaT,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consensus_text_90345944,,,,,,,14,NaT,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consensus_text_90345945,,,,,,,14,NaT,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
pd.options.display.max_colwidth=None
df_topics_by_sender = df[['sender']].join(doc_topic_scores_df).groupby('sender').mean(numeric_only=True)
df_topics_by_sender

df_cmp = pd.DataFrame()
df_cmp['mary'] = df_topics_by_sender.T['Mary Shackleton Leadbeater'].sort_values(ascending=False)
df_cmp['sarah'] = df_topics_by_sender.T['Sarah Leadbeater Barrington'].sort_values(ascending=False)
df_cmp['mary - sarah'] = df_cmp['sarah'] - df_cmp['mary']
df_cmp['topic'] = [' '.join(x for x,y in mdl.get_topic_words(i, 25)) for i in df_cmp.index]
df_cmp.sort_values('mary - sarah')

,mary,sarah,mary - sarah,topic
4,0.040258,0.002835,-0.037423,shall alas many said hear good like went agreeable room might eben want next bed remains guinea friends indeed child attention seemed wanting days gloom
19,0.036394,0.000336,-0.036058,heart tears address walker fanny attempt bees fruit garden match formed alexander consider beloved dawson side heaven basket accompanied upon partly kelly shop wont higher
36,0.024483,0.000485,-0.023998,good tho' letter bathing till tongue heart wil harper left upon pleasure even webb much eloquent little tooth irishtown car health enough countenance ache present
41,0.147593,0.124946,-0.022646,day home got came morning went last well mother see time aunt evening going night yesterday left also since two come could week little told
37,0.024650,0.002539,-0.022111,poor grief say old away joy went heart come first comes man times tears side friends fall soft philip black eye gay dined cheer i'm
45,0.033092,0.012720,-0.020372,dear father love sister mother feel sweet wish could dearest ever tho' friend hope must heart comfort beloved pain gone loved anxious comfortable never indeed
42,0.044003,0.029300,-0.014703,little much without yet many might time part believe present upon also found way subject every friends matter manner rather even perhaps though say would
11,0.071025,0.057324,-0.013702,thy dear well love cousin sister thee family much hope letter kind wife son children good health last daughter hear father affectionate mother brother friends
15,0.086164,0.074863,-0.011301,thou thee thy dear love hast wilt art would thine know friend thyself let give hand sent affectionate like farewell pleased yet believe glad nancy
40,0.017820,0.006543,-0.011277,friend felt loss dear may yet seemed heart death beloved account poor many near peace one friends feelings situation trying mind sympathy end little bear
